Прогноз спроса.
Тестовое задание для 1221Systems
Работу выполнил Батутин Андрей   
27.01.2025  

Задание:
Построить алгоритм прогнозирования спроса на 1 неделю вперёд, используя предоставленные данные. В качестве тестовой выборки использовать последний месяц из файла sales.csv. При решении необходимо подобрать метрику для оценки результатов.

# Импорты

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import njit
from ydata_profiling import ProfileReport
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
import catboost
import warnings
import optuna
warnings.filterwarnings('ignore')


pd.set_option('display.float_format','{:.3f}'.format) 
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 35)

# Функции

In [2]:
def add_weekend_and_holidays(data:pd.DataFrame, date_column='date'):
    from workalendar.europe import Russia
    """
    Добавляет признаки выходных и праздников для России.
    
    :param data: DataFrame с колонкой дат.
    :param date_column: Название колонки с датами.
    :return: DataFrame с добавленными колонками `is_weekend` и `is_holiday`.
    """
    cal = Russia()
    
    data['is_holiday'] = data[date_column].apply(lambda x: cal.is_holiday(x))
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['day'] = data['date'].dt.day
    data['day_of_week'] = data['date'].dt.dayofweek  
    data['week_of_year'] = data['date'].dt.isocalendar().week.astype(int)
    data['quarter'] = data['date'].dt.quarter
    data['is_weekend'] = data['date'].dt.dayofweek > 4
    data['is_month_start'] = data['date'].dt.is_month_start
    data['is_month_end'] = data['date'].dt.is_month_end
    data['season'] = data['date'].dt.month % 12 // 3 + 1  # 1-зима, 2-весна и т.д.
    
    return data

def check_quality(data:pd.DataFrame):
    """
    Функция проверяет качество данных.Используется множество раз
    """
    print('кол-во пропусков',data.isna().sum().sum())
    print('кол-во дубликатов',data.duplicated().sum())
    print('форма данных',data.shape)

def add_rolling_sum(data:pd.DataFrame, window_days:int, col_name:str):
    """
    Функция подсчета суммы продаж за N дней
    """
    data[col_name] = (
        data.groupby(['item_id', 'store_id'], group_keys=False)
        .apply(lambda group: group.rolling(
            window=f'{window_days}D', 
            on='date', 
            closed='left'  
        )['quantity'].sum())
        .reset_index(drop=True)
    )
    return data.fillna({col_name: 0})
def calculate_wape(y_true:pd.DataFrame, y_pred:pd.DataFrame):
    """
    Метрика качества WAPE
    """
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

def calculate_bias(y_true:pd.DataFrame, y_pred:pd.DataFrame):
    """
    Метрика качества BIAS
    Вычисляет смещение между мат ожиданием предсказания модели и фактом
    """
    return np.mean(y_pred - y_true)


# Функция с использованием Numba для ускорения
@njit
def custom_rolling_sum_numba(dates:pd.Timestamp, values:pd.DataFrame, window_days:int):
    """
    Кастомная функция для расчета суммы в окне из window_days дней вперед.
    Ускорена с помощью Numba.
    Создана для подсчета прогноза-факта на неделю вперед в тесте
    """
    result = np.zeros(len(dates))
    for i in range(len(dates)):
        current_date = dates[i]
        end_date = current_date + window_days
        window_sum = 0.0
        for j in range(i, len(dates)):
            if dates[j] < end_date:
                window_sum += values[j]
            else:
                break
        result[i] = window_sum
    return result

def add_sum(data:pd.DataFrame, col_name:str,col_pred:str ,window_days=7):
    """
    Кастомная оконная функция, группирует нужное окно,с учетом пропусков в данных
    """
    data['date'] = pd.to_datetime(data['date'])
    data = data.sort_values(by=['store_id', 'item_id', 'date'])
    
    # Применяем кастомный rolling для каждой группы
    data[col_name] = (
        data.groupby(['store_id', 'item_id'], group_keys=False)
        .apply(lambda group: pd.Series(
            custom_rolling_sum_numba(
                group['date'].values.astype(np.int64) // 10**9,  
                group[col_pred].values,                        
                pd.Timedelta(days=window_days).value // 10**9   
            ),
            index=group.index
        ))
        .reset_index(level=0, drop=True)
    )
    
    return data


MONTHS_PREDICT = pd.DateOffset(months=1)#константа.Месяц предсказания

# Исследовательский анализ данных

Получена дополнительная информация о тестовом задании от Екатерины (@katesib):  
Доля онлайн-заказов невелика, заказы формируются из остатков магазина, а не из дарксторов.  
Таким образом, заказы можно объединить и использовать их в дальнейшем совместно.

In [3]:
sales = pd.read_csv('sales.csv').drop('Unnamed: 0',axis=1)
online = pd.read_csv('online.csv').drop('Unnamed: 0',axis=1)
display(sales.columns)
display(online.columns)

Index(['date', 'item_id', 'quantity', 'price_base', 'sum_total', 'store_id'], dtype='object')

Index(['date', 'item_id', 'quantity', 'price_base', 'sum_total', 'store_id'], dtype='object')

Поскольку структура данных идентична, мы просто объединим их вертикально.

In [4]:
len(online) + len(sales) == pd.concat([online] + [sales]).shape[0]


True

In [5]:
sales = pd.concat([online] + [sales])

In [6]:
check_quality(sales)

кол-во пропусков 0
кол-во дубликатов 9
форма данных (8556097, 6)


Появились дубликаты

In [7]:
sales.loc[(sales['item_id']=='b58b26727fa4')&
          (sales['quantity']==1)&
          (sales['date']=='2023-06-09')]

,date,item_id,quantity,price_base,sum_total,store_id
812683,2023-06-09,b58b26727fa4,1.000,75.900,75.900,1
3385183,2023-06-09,b58b26727fa4,1.000,75.900,75.900,1


После слияния онлайн и офлайн данных возникло 9 дубликатов, хотя в отдельных данных их не наблюдается.  
Вероятно, существует ошибка в процессе записи и хранения чеков. Я удалю эти дубликаты.

In [8]:
sales = sales.drop_duplicates().reset_index(drop=True)
sales.shape


(8556088, 6)

In [9]:
8556097 - 9

8556088

In [10]:
sales.loc[(sales['date']=='2023-08-04')&(sales['item_id']=='f0309b5a974b')]

,date,item_id,quantity,price_base,sum_total,store_id
817,2023-08-04,f0309b5a974b,2.000,51.810,103.620,1
1123422,2023-08-04,f0309b5a974b,1.000,59.900,59.900,1


Отметим,что данные не агрегированы до конца до 1 дня по магазину,дате  
Проводем агрегацию  
Дополнительным плюсом будет сокращение размерности данных на 1млн записей

In [11]:
sales = sales.groupby(['date','item_id','store_id'])[['quantity','price_base']].agg({'quantity':'sum','price_base':'mean'}).reset_index()

In [12]:
check_quality(sales)

кол-во пропусков 0
кол-во дубликатов 0
форма данных (7649494, 5)


Подгрузим остальные данные

In [13]:

markdowns = pd.read_csv('markdowns.csv').drop('Unnamed: 0',axis=1)
discounts_history = pd.read_csv('discounts_history.csv').drop('Unnamed: 0',axis=1)
actual_matrix = pd.read_csv('actual_matrix.csv').drop('Unnamed: 0',axis=1)
catalog = pd.read_csv('catalog.csv').drop('Unnamed: 0',axis=1)
stores = pd.read_csv('stores.csv').drop('Unnamed: 0',axis=1)
price_history = pd.read_csv('price_history.csv').drop('Unnamed: 0',axis=1) 



История цен тоже содержит дубликаты

In [14]:
check_quality(price_history)
price_history.loc[(price_history['item_id']=='c0a8dcca7cdb')&(price_history['price']==229.9)&(price_history['date']=='2023-08-16')]


кол-во пропусков 0
кол-во дубликатов 18641
форма данных (698626, 5)


,date,item_id,price,code,store_id
6712,2023-08-16,c0a8dcca7cdb,229.900,11,1
6713,2023-08-16,c0a8dcca7cdb,229.900,11,1


должно остатся строк

In [15]:
698626 - 18641

679985

In [16]:
price_history = price_history.drop_duplicates().reset_index(drop=True)
price_history.shape

(679985, 5)

дубликаты удалены верно

Изучим данные с помощью pandas profiling.  
В этой секции не будет графиков.
Я изучу все таблицы по колоночно и сделаю соответствующие записи  
Запишу сразу то,что имеет отношение к подготовке данных.

In [17]:
ProfileReport(sales).to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

## Выводы по исследовательскому анализу данных,на что обратить внимание при подготовке данных



sales,online - продажи:  
- данные за 2 года продаж, 4 магазина
- Высокая корреляция между  price_base , sum_total 
- Сильно скошенные распределения влево у quantity,sum_total  
- Отрицательные значения в quantity , price. Это могут быть возвраты товаров покупателей
-  Выбросы в максимальх значениях quantity , price  

price_history  - история цен:
- price содержит 0 и макс цена 118496741
- price становится 0 по 19 разным type code,врядли это списания для нужд магазинов
- price  0 в price history 60421 / 679985 случаев.
- price 0 был для 20к товаров из 220к каталога



markdowns - скидки:  
- дубликаты 268
 - normal_price сильно коррелирует с price
- normal_price содержит 0
- скидки не для всех магазинов

discounts_history - история скидок:  
- с 2022-08-28 по 2045-12-31 даты(оверал)
- имеются пропуски в колонке promo_type_code
- сильная корреляции  между  sale_price_before_promo и sale_price_time_promo
- sale_price_before_promo содержит 0 и аномалии по цене до 17к
- sale_price_time_promo содержит 0 и аномалии по цене до 16к
- зачем нужен  promo_type_code 
- странные значения number_disc_day 8к дней  

actual_matrix - Актуальная матрица:  
- Даты  Minimum 2019-10-17 00:00:00 Maximum 2024-09-26


catalog - каталог продукции и ее характеристики:
- Weight_netto в целом сильно коррелирует с Weight_volume и fattnes
- item_type имеет  (80,2%) пропущенных значений.
- Weight_volume имеет  (62,3%) пропущенных значений, отрицательные значения
- Weight_netto имеет  (77,7%) пропущенных значений
- fatness имеет  (96,7%) пропущенных значений
- Weight_volume сильно искажен 
- Weight_netto сильно асимметричен  

stores - магазины:
- заметок нет


# Подготовка данных

В этой секции описываю то,что было сделано итого.  
Внутри каждой ячейки решение и рассуждение

## Таблица sales

sales:
- удалим все цены 0
- удалим sum_total,из-за высокой корреляции с price_base
- удалим отрицательные кол-ва покупок(возвраты покупатей.Всего 1к строк)
- удалять,изменять значения price_base по максимальному значению не будем.  
Так как все "аномальные" значения цен пренадлежат небольшой группе товаров и она является их реальной стоимостью,а не аномалией в данных
- удалять,изменять значения quantity по максимальному значению не будем.  
Так как все "аномальные" значения цен пренадлежат товарам с разной мерностью измерения,либо пакетам

In [18]:
print('цены <= 0 занимают ',np.round(sales.loc[sales['price_base']<=0].shape[0] / len(sales) * 100,2),'%','строк')
sales = sales.loc[sales['price_base']>0].reset_index(drop=True)
print('quantity <= 0 занимают ',np.round(sales.loc[sales['quantity']<0].shape[0] / len(sales) * 100,2),'%','строк')
sales = sales.loc[sales['quantity']>0].reset_index(drop=True)

цены <= 0 занимают  0.1 % строк
quantity <= 0 занимают  0.01 % строк


Можно удалить смело такие некачественные данные.  
Их не так много

In [19]:
sales.price_base.describe()

count   7640687.000
mean        203.347
std         331.374
min           0.010
25%          59.900
50%         109.000
75%         199.900
max       28999.900
Name: price_base, dtype: float64

In [20]:
sales.loc[sales['price_base']>20000]['item_id'].value_counts()

item_id
33dd4df7022d    16
Name: count, dtype: int64

Большие цены не являются аномалиями сбора данных.Это явно дорогой товар сам по себе.Нормальный коньяк за 20к

In [21]:
catalog.loc[catalog['item_id']=='33dd4df7022d']

,item_id,dept_name,class_name,subclass_name,item_type,weight_volume,weight_netto,fatness
25729,33dd4df7022d,КОНЬЯК,ИМПОРТ,КОНЬЯК,Коньяк,0.700,0.700,NaN


In [22]:
sales.quantity.describe()

count   7640687.000
mean          5.812
std          27.568
min           0.002
25%           1.000
50%           2.000
75%           5.000
max        5023.000
Name: quantity, dtype: float64

In [23]:
sales.loc[sales['quantity']>3000].head()

,date,item_id,store_id,quantity,price_base
631551,2022-11-19,b0d24502fb66,1,4656.000,6.970
944568,2022-12-29,b0d24502fb66,1,3008.000,6.890
954310,2022-12-30,b0d24502fb66,1,4524.000,6.920
963443,2022-12-31,b0d24502fb66,1,3907.000,6.920
1267315,2023-02-10,6d284b4e9982,1,4465.000,51.535


In [24]:
display(catalog.loc[catalog['item_id']=='b0d24502fb66'])
display(catalog.loc[catalog['item_id']=='6d284b4e9982'])

,item_id,dept_name,class_name,subclass_name,item_type,weight_volume,weight_netto,fatness
48581,b0d24502fb66,ВСПОМОГАТЕЛЬНАЯ ГРУППА,ПЛАТНЫЕ ПАКЕТЫ,ПЛАТНЫЕ ПАКЕТЫ,Упаковочный Материал,NaN,NaN,NaN


,item_id,dept_name,class_name,subclass_name,item_type,weight_volume,weight_netto,fatness
175220,6d284b4e9982,САХАР,САХАР-ПЕСОК,РАФИНИРОВАННЫЙ,Белый,1.000,1.000,NaN


с кол-во точно так же.  
Продажи пакетов агрегируются.  
Плюс у нас могут быть разные формы измерений(грам,шт,килограмм)
Не буду вырезать.  
Таблица sales готова

## Таблица price_history

price_history:
- удалим цены 0 (10% данных)
- удалим колонку code
- удалим артикулы,которых нет в sales
- удалим выброс с ценой 15690156


In [25]:
price_history = price_history.loc[price_history['price']>0].reset_index(drop=True)
price_history = price_history.drop('code',axis=1)
price_history = price_history.loc[price_history['item_id'].isin(sales['item_id'])]



In [26]:
price_history.price.describe()

count     598649.000
mean         359.980
std        20286.094
min            0.010
25%           85.900
50%          170.000
75%          399.000
max     15690156.400
Name: price, dtype: float64

In [27]:
price_history.sort_values('price',ascending=False)

,date,item_id,price,store_id
387990,2022-09-23,75fe6dc46c61,15690156.400,2
484570,2024-06-28,33dd4df7022d,26490.000,3
122450,2024-06-28,33dd4df7022d,26490.000,1
364348,2024-06-28,33dd4df7022d,26490.000,2
361790,2024-06-11,33dd4df7022d,25990.000,2
...,...,...,...,...
236105,2023-03-10,762109f0867a,0.010,1
489586,2024-08-30,19fb164683ba,0.010,3
489588,2024-08-30,3fae97cfe183,0.010,3
489589,2024-08-30,6c1060e74964,0.010,3


цена 26490 похожа на ноньяки что я видел сверху,а 15690156 нет.  
Эту строку удалю.  
Таблица price_history готова

In [28]:
price_history = price_history.drop(index=[387990]).reset_index(drop=True)

In [29]:
price_history.price.describe()

count   598648.000
mean       333.771
std        561.658
min          0.010
25%         85.900
50%        170.000
75%        399.000
max      26490.000
Name: price, dtype: float64

## Таблица markdown

markdowns:
- убедился в том,что таблица несодержит не нужных нам артикулов и дат
- удалил normal price 0
- удалил дубликаты


In [30]:

markdowns.loc[~markdowns['item_id'].isin(sales['item_id'])]

,date,item_id,normal_price,price,quantity,store_id


Присутствуют только те артикулы,что нужны

In [31]:
sales.date.min() , sales.date.max() 

('2022-08-28', '2024-09-26')

In [32]:
markdowns.date.min() , markdowns.date.max()

('2022-08-28', '2024-09-26')

Лишних дат нет,для теста имеются нужные скидки(промо)

нужна ли информация - продано со скидкой?

In [33]:
markdowns.normal_price.describe()

count   8979.000
mean     358.353
std      220.119
min        0.000
25%      189.000
50%      239.000
75%      549.000
max     2790.000
Name: normal_price, dtype: float64

In [34]:
markdowns.loc[markdowns['normal_price']==0]

,date,item_id,normal_price,price,quantity,store_id
4889,2024-05-20,8b7268eb2b33,0.000,299.100,2.000,2
8262,2024-05-20,8b7268eb2b33,0.000,349.000,1.000,4


Удалим цену normal_price == 0

In [35]:
markdowns = markdowns.drop(index=[4816,7994]).reset_index(drop=True)

In [36]:
markdowns.price.describe()

count   8977.000
mean     213.294
std      141.802
min       12.500
25%      100.000
50%      150.000
75%      330.000
max     1380.500
Name: price, dtype: float64

In [37]:
markdowns.quantity.describe()

count   8977.000
mean       3.071
std        4.237
min        0.115
25%        1.000
50%        2.000
75%        4.000
max      120.000
Name: quantity, dtype: float64

In [38]:
markdowns.duplicated().sum() , len(markdowns)

(268, 8977)

In [39]:
markdowns = markdowns.drop_duplicates().reset_index(drop=True)
markdowns.shape

(8709, 6)

In [40]:
8979 - 8711

268

Дубликаты удалены

таблица markdowns готова

## Таблица discount_history

discounts_history:
- удалил скидки по датам до конца теста
- удалил колонки promo_type_code ,doc_id
- удалил sale_price_before_promo ==  0 и sale_price_time_promo == 0
- удалил ситуации ,когда цена после промо становится либо выше либо не изменяется.
- Удалил строки,со сроками проведения промо выше 42 дней( борьба с аномалиями)

In [41]:
discounts_history = discounts_history.loc[discounts_history['date']<'2024-09-26'].reset_index(drop=True)


In [42]:
discounts_history.shape

(3426146, 8)

In [43]:
discounts_history = discounts_history.drop(['promo_type_code','doc_id'],axis=1)

Есть аномалии в таблице,где цена во время промо\после промо равна 0.
Либо во время промо увеличивается

In [44]:
discounts_history.loc[discounts_history['sale_price_before_promo']==0].shape

(21039, 6)

In [45]:
discounts_history = discounts_history.loc[discounts_history['sale_price_before_promo']>0].reset_index(drop=True)

21039 строк содержат 0

In [46]:
discounts_history.loc[discounts_history['sale_price_time_promo']==0]

,date,item_id,sale_price_before_promo,sale_price_time_promo,number_disc_day,store_id
2549428,2023-10-30,b60c43f065bf,184.900,0.000,1.000,3


In [47]:
discounts_history = discounts_history.drop(index=[2549428]).reset_index(drop=True)

In [48]:
discounts_history['number_disc_day'].describe()

count   3405106.000
mean         18.876
std          78.995
min           1.000
25%           4.000
50%           8.000
75%          12.000
max        1091.000
Name: number_disc_day, dtype: float64

что же за странные 173 и выше дней проведения промо

In [49]:
discounts_history.loc[discounts_history['number_disc_day']>173]

,date,item_id,sale_price_before_promo,sale_price_time_promo,number_disc_day,store_id
165,2022-08-28,8250099a8aff,59.900,59.900,240.000,1
166,2022-08-28,23b18d1f7da9,59.900,59.900,240.000,1
167,2022-08-28,e5b2b6f57ea7,59.900,59.900,240.000,1
168,2022-08-28,2c007023f650,59.900,59.900,240.000,1
229,2022-08-28,9f716dce9a57,59.900,59.900,240.000,1
...,...,...,...,...,...,...
3404722,2024-09-25,367ade44c168,159.900,159.900,246.000,4
3404723,2024-09-25,af4a7cdfc6aa,399.900,399.900,246.000,4
3404724,2024-09-25,0ea52050198d,399.900,399.900,246.000,4
3404725,2024-09-25,c22182ff43cf,399.900,399.900,246.000,4


Видим странные промо,где цена не меняется.Удалим аномалии.  
Удалим строки,где применяется промо.А цена становится выше

In [50]:
discounts_history.shape

(3405106, 6)

In [51]:
discounts_history.loc[discounts_history['sale_price_before_promo']<=discounts_history['sale_price_time_promo']]

,date,item_id,sale_price_before_promo,sale_price_time_promo,number_disc_day,store_id
25,2022-08-28,68348aae54c8,99.900,99.900,6.000,1
123,2022-08-28,aa6d09435319,799.900,799.900,6.000,1
135,2022-08-28,7428830d55b6,279.900,279.900,6.000,1
165,2022-08-28,8250099a8aff,59.900,59.900,240.000,1
166,2022-08-28,23b18d1f7da9,59.900,59.900,240.000,1
...,...,...,...,...,...,...
3405067,2024-09-25,fb282818b8eb,149.900,149.900,14.000,4
3405076,2024-09-25,4df0b1a72540,849.900,899.900,3.000,4
3405092,2024-09-25,7daa478b4b52,599.900,599.900,3.000,4
3405093,2024-09-25,e199dccdd490,1499.900,1599.900,3.000,4


In [52]:
discounts_history = discounts_history.loc[discounts_history['sale_price_before_promo']>discounts_history['sale_price_time_promo']]

In [53]:
discounts_history.shape

(2785691, 6)

Остается 1091 день проведения промо

In [54]:
discounts_history.number_disc_day.describe()

count   2785691.000
mean         16.784
std          69.723
min           1.000
25%           4.000
50%           8.000
75%          12.000
max        1091.000
Name: number_disc_day, dtype: float64

In [55]:
display(discounts_history.number_disc_day.quantile(0.98))
display(discounts_history.number_disc_day.quantile(0.99))

29.0

436.0

Существует значительный перекос между 98 и 99 квантилем.  
Я планирую оставить месяц для проведения промо-акции.  
При этом я исхожу из предположения, что не располагаю точными данными о сроках применения промо. 
Предполагаю, что все значения, превышающие 42 дня, могут быть связаны с ошибками в хранении и обработке данных.   
Число 42 выбрано, потому что я могу допустить, что промо-акция была рассчитана на три этапа поставки (по три поставки по две недели каждая).

In [56]:
discounts_history = discounts_history.loc[discounts_history['number_disc_day']<42].reset_index(drop=True)

In [57]:
discounts_history.number_disc_day.describe()

count   2734769.000
mean          8.214
std           5.096
min           1.000
25%           4.000
50%           8.000
75%          12.000
max          41.000
Name: number_disc_day, dtype: float64

Таблица discounts_history готова

## Таблица actual matrix

actual matrix:
- ничего делать не буду,так как из нее нужно только бинарный признак подгрузить


даты резать не нужно

## Таблица catalog

catalog:
- востановил каскадным групповым средним коробочный обьем товаров
- удалил все колонки кроме айди и обьема коробки товара


In [58]:
group_columns = ['dept_name', 'class_name', 'subclass_name', 'item_type']
for col in group_columns:
    catalog['weight_volume'] = catalog['weight_volume'].fillna(
        catalog.groupby(col)['weight_volume'].transform('median')
    )

Заполним каскадным (потому что категории по разному имеют пропуски) медианой по категории признак обьем товара

In [59]:
catalog.loc[catalog['weight_volume'].isna()]['dept_name'].value_counts()

dept_name
ГРИЛЬ                                    69
ПОДАРОЧНЫЕ КОРЗИНЫ                       43
РЫБНАЯ КУЛИНАРИЯ                         26
КОЛБАСНАЯ ГАСТРОНОМИЯ НЕ ИСПОЛЬЗОВАТЬ     2
Name: count, dtype: int64

Остаточные крохи строк(из 220к) заполним медианой

In [60]:
catalog['weight_volume'] = catalog['weight_volume'].fillna(catalog['weight_volume'].median())

In [61]:
catalog = catalog[['item_id','dept_name','weight_volume']]

In [62]:
catalog.weight_volume.isna().sum()

0

In [63]:
catalog.head()

,item_id,dept_name,weight_volume
0,da17e2d5feda,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,150.000
1,614de2b96018,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,30.000
2,0c1f1f3e3e11,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,15.000
3,71a7fa99f005,ТОВАРЫ ДЛЯ ДОМА,0.800
4,ec1bd4d59fe9,БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ,30.000


Таблица catalog готова

Итоги предобработки данных:
- проверены таблицы на наличие:
    - пропусков
    - дубликатов
    - аномалий
- проведена работа по :
    - удалению неинформативных колонок
    - удаление коррелирующих колонок
    - неиспользуемых строк данных  

Обьеденим данные из разных таблиц в 1 набор данных  


# Обьеденение данных

Что произведено:
- обьеденили sales с историей скидок
- markdowns использовать не буду так как не полна.
- price history не полна. Не будем использовать
- actual matrix очень не полна.Мало данных можем извлечь при конктатенации данных


Далее общая переменная будет называться train до этапа разбиения на выборки

In [64]:
train = sales.merge(discounts_history,on=['date','item_id','store_id'],how='left')


In [65]:
check_quality(train)

кол-во пропусков 19221255
кол-во дубликатов 0
форма данных (7640687, 8)


Обьедим историю скидок

Получили разряженную таблицу 

In [66]:

train.loc[train['sale_price_before_promo'].notna()].shape

(1233602, 8)

заполнили 1.3млн строк информации о проведении промо


In [67]:
train.loc[train['sale_price_before_promo'].notna()].sample(5)


,date,item_id,store_id,quantity,price_base,sale_price_before_promo,sale_price_time_promo,number_disc_day
2180804,2023-06-06,b3606eafdc44,1,0.104,2999.040,3999.000,2999.000,13.000
6079966,2024-06-07,5d03f75223ba,4,6.000,120.850,129.900,119.900,2.000
2455275,2023-07-11,deb52f80639e,1,1.000,61.260,99.900,89.900,6.000
338594,2022-10-12,918bd27caa8c,2,2.000,279.900,329.900,279.900,16.000
6343426,2024-06-26,4346ab99efcb,4,2.000,25.900,34.900,25.900,7.000


Где то price base совпадает с промо,где то нет.  
Но мы и не знаем на каком окне вычислялся price_base

In [68]:
train.shape

(7640687, 8)

In [69]:
train = train.merge(catalog,on='item_id',how='left')

In [70]:
check_quality(train)

кол-во пропусков 19295999
кол-во дубликатов 0
форма данных (7640687, 10)


In [71]:
train.loc[train['weight_volume'].isna()].sample()

,date,item_id,store_id,quantity,price_base,sale_price_before_promo,sale_price_time_promo,number_disc_day,dept_name,weight_volume
7020109,2024-08-13,8d20185399b4,2,1.000,129.900,NaN,NaN,NaN,NaN,NaN


In [72]:
catalog.loc[catalog['item_id']=='e35f0201fc8c']

,item_id,dept_name,weight_volume


В каталогах нет некоторых артикулов

Выводы по качеству данных:
- имеются пропуски. Закодируем неизвестной кат.переменной.В реальной работе нужно добыть чистые данные
- нет информации об остатках в магазине.Это очень плохо.
- Нет прогноза погоды.Я бы припарсил погоду,но нет информации о гео
- Нет оценки покупательского спроса
- Нет профиля магазинов по клиентам(но я настойчивый)
- нет срока годности товаров
- из за не полной матрицы актуальных товаров не можем получить бинарный признак вхождения в ассортимент на дату или нет
- нет информаций о магазинах конкурентов и расстоянии до них

Таблицы discounts histrory, catalog , sales обьеденили  
Приступим к генерации фичей

# Генерация фичей

Фичи которые уже сгенерированы присоеденением таблиц:
- информация об ожидаемом промо  
Лист генерируемых фичей:
- продажи за прошлую неделю,2,3,4
- признаки по дате (номер недели в году,сезон,квартал,номер дня в недели)
- профиль магазина


генерируем признак : сумма продаж за 7,14,21,28 дней

In [73]:
train['date'] = pd.to_datetime(train['date'])
train = train.sort_values(by=['item_id', 'store_id', 'date']).reset_index(drop=True)


for weeks, days in [(1,7), (2,14), (3,21), (4,28)]:
    train = add_rolling_sum(train, days, f'sales_last_{weeks}_weeks')

Какая была продажа N времени назад

In [74]:
train.loc[(train['item_id']=='4aa8dbe05246')&(train['date']=='2023-08-04')]

,date,item_id,store_id,quantity,price_base,sale_price_before_promo,sale_price_time_promo,number_disc_day,dept_name,weight_volume,sales_last_1_weeks,sales_last_2_weeks,sales_last_3_weeks,sales_last_4_weeks
2252460,2023-08-04,4aa8dbe05246,1,65.000,25.505,NaN,NaN,NaN,ХЛЕБ,0.300,451.000,901.000,1350.000,1835.000


Проверим  ручками.Фичи правильно посчитаны

Признак , сколько продаж было N времени назад

In [75]:

for lag in [7, 14,30]:
    train[f'quantity_lag_{lag}'] = train.groupby(['item_id', 'store_id'])['quantity'].shift(lag)

Добавим информацию по дате(день недели,номер недели,года,сезона,выходные,праздники и тд)

In [76]:
train = add_weekend_and_holidays(train)

In [77]:
train['is_holiday'].value_counts()

is_holiday
False    7359776
True      280911
Name: count, dtype: int64

Проверил,совпадает по производственному календарю

Признак абсолютная скидка и скидка в процентах

In [78]:
train['discount_price_perc'] = (train['sale_price_time_promo'] - train['sale_price_before_promo'])/train['sale_price_before_promo']*100
train['discount_price_abs'] = train['sale_price_time_promo'] - train['sale_price_before_promo']
train = train.drop(['sale_price_before_promo','sale_price_time_promo'],axis=1) #удалю неиспользуемую колонку

In [79]:
SPLIT_DATE = pd.to_datetime(train['date'].max()) - MONTHS_PREDICT

features = ['day', 'day_of_week','quarter',]

for feature in features:                       
    avg_sales = train.loc[pd.to_datetime(train['date'])<SPLIT_DATE].groupby(['store_id', feature])['quantity'].mean().reset_index()
                                                        #^^^^ вот тут мы не допустили утечку
    avg_sales.rename(columns={'quantity': f'{feature}_avg_sales'}, inplace=True)
    
    train = train.merge(avg_sales, on=['store_id', feature], how='left')

Мержим обратно  
Мы расчитали сезонность на основе трейна.  
Замержили обратно.  
Получается,что эти фичей для теста буду расчитаны значения на основе трейна как и в жизни

Я планирую создать быстрый профиль магазина на основе данных о клиентах.  
В первую очередь, я извлеку категории товаров и передам их в модель DeepSeek.  
Она самостоятельно рассортирует товары по категориям: 18+, готовая еда, продукты и товары для дома.  
После этого я проверю, насколько качественно выполнена кластеризация, и добавлю соответствующую функциональность.

In [80]:
train['dept_name'].sample(3)

67503      СУХИЕ КОМПОНЕНТЫ
4852828        СЫРЫ ШТУЧНЫЕ
4429346        СЫРЫ ВЕСОВЫЕ
Name: dept_name, dtype: object

In [81]:
#DEEPSEEK
# https://chat.deepseek.com/
# Товары для совершеннолетних (алкоголь и закуски к нему)
#Кластеризация дипсика
tovary_dlya_sovershennoletnih = pd.Series(['ПИВО','КОНЬЯК','СУХАРИКИ','ВИНО','ЧИПСЫ','ХОЛОДНЫЕ ЗАКУСКИ','РЫБКА К ПИВУ',
       'СЕМЕЧКИ','СОСИСКИ.САРДЕЛЬКИ.ШПИКАЧКИ','ИКРА РЕЧНЫХ И МОРСКИХ РЫБ','ГРИБЫ','ОРЕХИ','ВОДКА,НАСТОЙКИ','СНЕКИ',
       'КРЕПКИЕ АЛКОГОЛЬНЫЕ НАПИТКИ','МЯСНЫЕ СНЕКИ','СЛАБОАЛКОГОЛЬНЫЕ НАПИТКИ','СОЛЕНИЯ','ЭТНИЧЕСКИЕ СНЭКИ И ВОДОРОСЛИ',
       'МЯСНЫЕ ДЕЛИКАТЕСЫ','РЫБА СОЛЕНАЯ','РЫБА ХОЛОДНОГО КОПЧЕНИЯ','РЫБА ГОРЯЧЕГО КОПЧЕНИЯ','КОЛБАСЫ КОПЧЕНЫЕ.СЫРОВЯЛЕНЫЕ','КОЛБАСЫ ВАРЕНЫЕ'
                                       
])

# Готовая еда 
gotovay_food = pd.Series(['ПРОДУКЦИЯ БЫСТРОГО ПРИГОТОВЛЕНИЯ','ГОТОВЫЕ БЛЮДА','ЗАВТРАКИ','ПЕРВЫЕ БЛЮДА', 'ВТОРЫЕ БЛЮДА',
       'БЫСТРЫЙ ПЕРЕКУС','СУШИ','ПИЦЦА','ПЕЛЬМЕНИ,МАНТЫ,ХИНКАЛИ','САЛАТЫ','БУРГЕРЫ', 'ГОРЯЧИЕ БУТЕРБРОДЫ','ФРУКТОВЫЕ САЛАТЫ',
       'ПИЦЦА ПОЛУФАБРИКАТ','БУТЕРБРОДЫ','ШАУРМА','ПОЛУФАБРИКАТЫ','ВОК','ПОНЧИКИ','СЛОЙКА','СЛАДКИЕ МУЧНЫЕ ИЗДЕЛИЯ','ЛЕПИМ-ВАРИМ',
       'СДОБА','РЫБНЫЕ ПОЛУФАБРИКАТЫ','РЫБНАЯ КУЛИНАРИЯ','РЫБНЫЕ ПОЛУФАБРИКАТЫ ОХЛАЖДЕННЫЕ','ДОМАШНЯЯ КУХНЯ','МЯСНЫЕ П/Ф','ПИРОЖНЫЕ', 
       'ТОРТЫ И ПИРОЖНЫЕ','ПИРОГИ','ТОРТЫ','ИЗДЕЛИЯ ИЗ ТЕСТА','МОРОЖЕНОЕ И ЗАМОРОЖЕННЫЕ ДЕСЕРТЫ'
       'КАФЕ', 
       'СДОБНЫЕ ИЗДЕЛИЯ(НЕ ИСПОЛЬЗОВАТЬ)','ПРИКАССА', 'nan',
       'РЫБА СВЕЖАЯ НЕ ИСПОЛЬЗОВАТЬ','СПОРТИВНОЕ ПИТАНИЕ',
]) 

# Продукты
produkty = pd.Series(['ТРАДИЦИОННЫЕ МОЛОЧНЫЕ ПРОДУКТЫ', 'ДЕТСКИЕ МОЛОЧНЫЕ ПРОДУКТЫ','СОВРЕМЕННАЯ МОЛОЧНАЯ КАТЕГОРИЯ', 
       'КОНСЕРВЫ МОЛОЧНЫЕ','РАСТИТЕЛЬНЫЕ МОЛОЧНЫЕ ПРОДУКТЫ','ХЛЕБ','СОКИ','ДЕТСКОЕ ПИТАНИЕ','МОЛОКО','ОВОЩИ ЗАМОРОЖЕННЫЕ',
       'ЯЙЦО','ФРУКТЫ','ВОДА','КОНФЕТЫ','МУКА','ЗЕЛЕНЬ СВЕЖАЯ','ОВОЩИ','КОФЕ','ЧАЙ','МАСЛО ПОДСОЛНЕЧНОЕ','САХАР','СЛАДОСТИ', 
       'МАСЛО И МАРГАРИН','СЛИВКИ','КОНСЕРВЫ МОЛОЧНЫЕ','СОВРЕМЕННАЯ МОЛОЧНАЯ КАТЕГОРИЯ','СВЕЖЕЕ МЯСО','МЯСО,ПТИЦА ЗАМОРОЖЕННЫЕ',
       'ПТИЦА','МАСЛО ПРОЧЕЕ','УКСУС','СОКИ,МОРСЫ,НАПИТКИ','КВАС','ЧАЙ ХОЛОДНЫЙ','ЭНЕРГЕТИКИ','ФУНКЦИОНАЛЬНЫЕ НАПИТКИ','ТЕСТО',
       'ЛИМОНАДЫ', 'МАСЛО ОЛИВКОВОЕ', 'КРУПЫ И ЗЕРНОВЫЕ', 'ИНГРЕДИЕНТЫ','СИРОПЫ','СОЛЬ', 'МАКАРОННЫЕ ИЗДЕЛИЯ','НЕСЛАДКИЕ МУЧНЫЕ ИЗДЕЛИЯ', 
       'КОНСЕРВЫ МЯСНЫЕ','КОНСЕРВЫ РЫБНЫЕ','СУХОФРУКТЫ','ИНГРЕДИЕНТЫ ДЛЯ ПРИХОДА МЯСА','ПРОЧИЕ СОУСЫ','ВАРЕНЬЕ,МЁД', 'ПРОДУКТЫ ИЗ СУРИМИ',
       'СПЕЦИИ,ПРИПРАВА','СОУСЫ ДЛЯ ЭТНИЧЕСКОЙ КУХНИ И ПАСТЫ','КОНСЕРВЫ ГРИБНЫЕ','КОНСЕРВЫ ФРУКТОВЫЕ,ЯГОДНЫЕ','КОНСЕРВЫ ОВОЩНЫЕ',
       'ПРЕСЕРВЫ.ПАШТЕТЫ','СУХИЕ КОМПОНЕНТЫ','КОРЖИ И ТАРТАЛЕТКИ','СЫРЫ ВЕСОВЫЕ','СЫРЫ ШТУЧНЫЕ','ШОКОЛАД,ШОКОЛАДНАЯ ПАСТА', 
       'ЭТНИЧЕСКИЕ МАКАРОННЫЕ ИЗДЕЛИЯ', 'ДИЕТИЧЕСКОЕ ПИТАНИЕ','РАСТИТЕЛЬНЫЕ МОЛОЧНЫЕ ПРОДУКТЫ','ЭТНИЧЕСКАЯ КОНСЕРВАЦИЯ','ГРИБЫ ЗАМОРОЖЕННЫЕ', 
       'ЯГОДЫ ЗАМОРОЖЕННЫЕ','ИНГРЕДИЕНТЫ ДЛЯ ЭТНИЧЕСКОЙ КУХНИ','КЕТЧУПЫ,СОУСЫ НА ТОМАТНОЙ ОСНОВЕ', 'МАЙОНЕЗ,СОУСЫ НА МАЙОНЕЗНОЙ ОСНОВЕ',
       'ОСТРЫЕ СОУСЫ','РЫБА ЖИВАЯ','КРАСНАЯ РЫБА ОХЛАЖДЕННАЯ', 'БЕЛАЯ РЫБА МОРСКАЯ','КРАСНАЯ РЫБА','БЕЛАЯ РЫБА МОРСКАЯ ОХЛАЖДЕННАЯ',
       'БЕЛАЯ РЫБА РЕЧНАЯ','БЕЛАЯ РЫБА РЕЧНАЯ ОХЛАЖДЕННАЯ','МОРЕПРОДУКТЫ ОХЛАЖДЕННЫЕ','МОРЕПРОДУКТЫ','ИКРА ЛОСОСЕВАЯ', 'ИКРА БЕЛКОВАЯ', 
       'ИКРА РЫБ ОСЕТРОВЫХ ПОРОД','КАКАО,ШОКОЛАД','НАБОРЫ КОНФЕТ','КРЕВЕТКИ'

 
])

# Товары для дома
tovary_dlya_doma = pd.Series(['ТОВАРЫ ДЛЯ ДОМА','МАНГАЛ','ЛАКОМСТВА ДЛЯ ЖИВОТНЫХ','ТОВАРЫ ДЛЯ ШКОЛЫ И ОФИСА',
       'КОРМА ДЛЯ КОШЕК','ТОВАРЫ ДЛЯ УБОРКИ','ИНГРЕДИЕНТЫ ДЛЯ ПЕКАРНИ','НАПОЛНИТЕЛИ','ТОВАРЫ ДЛЯ ПРАЗДНИКА',
       'СРЕДСТВА ДЛЯ СТИРКИ','ДЕТСКИЙ МИР','КОРМА ДЛЯ ДРУГИХ ЖИВОТНЫХ','ЦВЕТЫ','СОПУТСТВУЮЩИЕ ТОВАРЫ ДЛЯ СТИРКИ',
       'ТОВАРЫ ДЛЯ ЛИЧНОГО ПОЛЬЗОВАНИЯ', 'ЗООТОВАРЫ','АКЦИЯ ЛОЯЛЬНОСТИ','ОСВЕЖИТЕЛИ,ИНСЕКТИЦИДЫ', 'АВТОТОВАРЫ',
       'КОРМА ДЛЯ СОБАК', 'ЧИСТЯЩИЕ,МОЮЩИЕ СРЕДСТВА','БУМАЖНО-ВАТНАЯ ПРОДУКЦИЯ','ЗОЖ','АКСЕССУАРЫ ДЛЯ КУРЕНИЯ', 
       'НАРОДНЫЕ ПРОМЫСЛЫ','ИНФОРМАЦИОННЫЕ ТОВАРЫ', 'ГРИЛЬ','ГЛОБАЛЬНЫЙ КАТАЛОГ','ПЕРСОНАЛЬНЫЙ УХОД','ТАБАЧНЫЕ ИЗДЕЛИЯ',
       'ВСПОМОГАТЕЛЬНАЯ ГРУППА', 'СЕЗОННЫЙ АССОРТИМЕНТ'
 
])


Сделаем профиль магазина по покупкам  
Так как мы не можем посчитать на профиль на основе данных теста - подход тот же.  
Вырезаем тест,а затем мёржим

In [82]:
train_for_profiling = train.loc[pd.to_datetime(train['date'])<SPLIT_DATE].reset_index(drop=True)[['store_id','dept_name']]

In [83]:

train_for_profiling['profile_18'] = train_for_profiling['dept_name'].apply(lambda row: 1 if row in tovary_dlya_sovershennoletnih.values else None)
train_for_profiling['profile_ready_foot'] = train_for_profiling['dept_name'].apply(lambda row: 1 if row in gotovay_food.values else None)
train_for_profiling['profile_products'] = train_for_profiling['dept_name'].apply(lambda row: 1 if row in produkty.values else None)
train_for_profiling['profile_home'] = train_for_profiling['dept_name'].apply(lambda row: 1 if row in tovary_dlya_doma.values else None)



In [84]:
train_for_profiling.sample()

,store_id,dept_name,profile_18,profile_ready_foot,profile_products,profile_home
6940175,4,ДЕТСКОЕ ПИТАНИЕ,NaN,NaN,1.000,NaN


заполним пропуски

In [85]:
train_for_profiling = train_for_profiling.fillna(0)

In [86]:
magazine_profile = train_for_profiling.groupby('store_id')[['profile_18','profile_ready_foot','profile_products','profile_home']].mean()

In [87]:
magazine_profile

,profile_18,profile_ready_foot,profile_products,profile_home
store_id,,,,
1,0.149,0.170,0.506,0.157
2,0.164,0.168,0.543,0.109
3,0.185,0.208,0.449,0.138
4,0.158,0.168,0.525,0.130


Определенная чувствительность есть в определенных срезах.Присоеденим как признак к магазину

In [88]:
train = train.merge(magazine_profile,on='store_id',how='left')
train.shape

(7640687, 35)

Заполнение пропусков,после создание признаков

# Заполнение пропусков

In [89]:
isna_columns = train.isna().sum()

In [90]:
isna_columns.loc[isna_columns!=0].index

Index(['number_disc_day', 'dept_name', 'weight_volume', 'quantity_lag_7',
       'quantity_lag_14', 'quantity_lag_30', 'discount_price_perc',
       'discount_price_abs'],
      dtype='object')

In [91]:
train['number_disc_day'] = train['number_disc_day'].fillna(0)

Если пропуск в проведении акции - значит акция не проводится.0

In [92]:
train['dept_name'] = train['dept_name'].fillna('Unknown')

Неизвестные категории в продукте проставим для пропусков.(Они являются новыми продуктами)

In [93]:
train['weight_volume'] = train['weight_volume'].fillna(train['weight_volume'].median())

Обьем коробок проставим как медиану по нашим продуктам

In [94]:
train['discount_price_perc'] = train['discount_price_perc'].fillna(0)
train['discount_price_abs'] = train['discount_price_abs'].fillna(0)

Если скидки нет,значит скидки 0 

In [95]:
train.shape

(7640687, 35)

Удалим пустые строки с лагами в 30 дней.Тк для них не можем расчитать значение.

In [96]:
train = train.dropna(subset=['quantity_lag_30'])

In [97]:
na = train.isna().sum()
na.loc[na!=0]

Series([], dtype: int64)

пропуски удалены

Делим данные.  
Тестовая выборка будет использоваться для оценки финального решения.  
Трейн будет использоваться для разбиения на трейн и валид и тест самого трейна  

In [98]:
SPLIT_DATE = pd.to_datetime(train['date'].max()) - MONTHS_PREDICT
test = train.loc[pd.to_datetime(train['date'])>=SPLIT_DATE].reset_index(drop=True).copy()
train = train.loc[pd.to_datetime(train['date'])<SPLIT_DATE].reset_index(drop=True).copy()

In [99]:
train.date.describe() , test.date.describe()

(count                          5885057
 mean     2023-11-11 02:45:57.514568192
 min                2022-09-27 00:00:00
 25%                2023-05-27 00:00:00
 50%                2023-12-17 00:00:00
 75%                2024-05-03 00:00:00
 max                2024-08-25 00:00:00
 Name: date, dtype: object,
 count                           413007
 mean     2024-09-10 11:04:25.288482048
 min                2024-08-26 00:00:00
 25%                2024-09-02 00:00:00
 50%                2024-09-10 00:00:00
 75%                2024-09-19 00:00:00
 max                2024-09-26 00:00:00
 Name: date, dtype: object)

Разбито верно

# Отбор признаков

Отберем важные признаки по рангам на кроссвалидация через time series splitter  
Определимся с метриками:
- лосс RMSE ,тк наиболее сильно штрафующий за сильные недопрогноз-перепрогноз
- метрика качества WAPE , BIAS , R2 , МАЕ


In [100]:
y = train['quantity']
train = train.drop('quantity',axis=1)
categorycal_cols = train.select_dtypes(include=['object']).columns.to_list()

In [106]:
train.to_parquet('train_without_target17')
pd.DataFrame(y).to_parquet('y_train17')

Будем собирать ключевые метрики на 5 фолдах time series splitter и важность признаков,измеренных в рангах

In [101]:
all_importances = pd.DataFrame() # для сбора рангов на итерациях
tscv = TimeSeriesSplit(n_splits=5)
val_ratio = 0.2
metrics = {
    'R2': [],
    'WAPE': [],
    'BIAS': [],
    'MAE': []
}
for _, (train_index, test_index) in enumerate(tscv.split(train)):
    train_train, train_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    split_idx = int(len(train_train) * (1 - val_ratio))
    train_train, train_val = train_train.iloc[:split_idx], train_train.iloc[split_idx:]
    y_train, y_val = y_train.iloc[:split_idx], y_train.iloc[split_idx:]
    
    model = catboost.CatBoostRegressor(random_seed=313,
                                   cat_features=categorycal_cols,
                                   loss_function='RMSE',
                                   early_stopping_rounds=50)
    model.fit(train_train, y_train , eval_set=(train_val, y_val),plot=True)
    y_pred = model.predict(train_test)
    metrics['R2'].append(r2_score(y_test, y_pred))
    metrics['WAPE'].append(calculate_wape(y_test, y_pred))
    metrics['BIAS'].append(calculate_bias(y_test, y_pred))
    metrics['MAE'].append(mean_absolute_error(y_test, y_pred))

    feature_importances = pd.DataFrame({'w':model.feature_importances_ , 'name':train.columns}).sort_values('w',ascending=False).reset_index(drop=True)
    feature_importances['index'] = len(feature_importances) - feature_importances.index
    all_importances = pd.concat([all_importances]+[feature_importances])

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.144998
0:	learn: 16.3968509	test: 10.0204500	best: 10.0204500 (0)	total: 148ms	remaining: 2m 28s
1:	learn: 14.7140905	test: 9.0704708	best: 9.0704708 (1)	total: 208ms	remaining: 1m 43s
2:	learn: 13.3143576	test: 8.3251979	best: 8.3251979 (2)	total: 267ms	remaining: 1m 28s
3:	learn: 12.1795231	test: 7.7884429	best: 7.7884429 (3)	total: 335ms	remaining: 1m 23s
4:	learn: 11.2388376	test: 7.3608942	best: 7.3608942 (4)	total: 392ms	remaining: 1m 18s
5:	learn: 10.4722466	test: 6.9949050	best: 6.9949050 (5)	total: 439ms	remaining: 1m 12s
6:	learn: 9.8529698	test: 6.7680793	best: 6.7680793 (6)	total: 491ms	remaining: 1m 9s
7:	learn: 9.3522630	test: 6.5345625	best: 6.5345625 (7)	total: 541ms	remaining: 1m 7s
8:	learn: 8.9432574	test: 6.3708099	best: 6.3708099 (8)	total: 585ms	remaining: 1m 4s
9:	learn: 8.5659534	test: 6.2618271	best: 6.2618271 (9)	total: 633ms	remaining: 1m 2s
10:	learn: 8.2885245	test: 6.1563293	best: 6.1563293 (10)	total: 693ms	remaining: 1m 2s
11:	lear

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.161668
0:	learn: 14.3792866	test: 9.8155364	best: 9.8155364 (0)	total: 97.4ms	remaining: 1m 37s
1:	learn: 12.7811001	test: 8.9623677	best: 8.9623677 (1)	total: 193ms	remaining: 1m 36s
2:	learn: 11.5099052	test: 8.3505402	best: 8.3505402 (2)	total: 293ms	remaining: 1m 37s
3:	learn: 10.4952196	test: 7.7812212	best: 7.7812212 (3)	total: 424ms	remaining: 1m 45s
4:	learn: 9.6869809	test: 7.3607812	best: 7.3607812 (4)	total: 530ms	remaining: 1m 45s
5:	learn: 9.0569067	test: 7.0414229	best: 7.0414229 (5)	total: 637ms	remaining: 1m 45s
6:	learn: 8.5664127	test: 6.8158541	best: 6.8158541 (6)	total: 757ms	remaining: 1m 47s
7:	learn: 8.1923409	test: 6.6463634	best: 6.6463634 (7)	total: 887ms	remaining: 1m 50s
8:	learn: 7.8862911	test: 6.5185704	best: 6.5185704 (8)	total: 1.03s	remaining: 1m 53s
9:	learn: 7.6463256	test: 6.3958218	best: 6.3958218 (9)	total: 1.16s	remaining: 1m 54s
10:	learn: 7.4692478	test: 6.3085204	best: 6.3085204 (10)	total: 1.27s	remaining: 1m 53s
11:	le

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.172294
0:	learn: 16.1174417	test: 18.0621310	best: 18.0621310 (0)	total: 186ms	remaining: 3m 5s
1:	learn: 14.0951272	test: 16.5308045	best: 16.5308045 (1)	total: 370ms	remaining: 3m 4s
2:	learn: 12.4981477	test: 15.4708412	best: 15.4708412 (2)	total: 530ms	remaining: 2m 56s
3:	learn: 11.2477840	test: 14.6476707	best: 14.6476707 (3)	total: 702ms	remaining: 2m 54s
4:	learn: 10.2749317	test: 14.1120936	best: 14.1120936 (4)	total: 855ms	remaining: 2m 50s
5:	learn: 9.5194280	test: 13.6646816	best: 13.6646816 (5)	total: 1.04s	remaining: 2m 52s
6:	learn: 8.9563826	test: 13.3299969	best: 13.3299969 (6)	total: 1.2s	remaining: 2m 50s
7:	learn: 8.5073196	test: 13.0782556	best: 13.0782556 (7)	total: 1.41s	remaining: 2m 54s
8:	learn: 8.1716011	test: 12.9129654	best: 12.9129654 (8)	total: 1.56s	remaining: 2m 52s
9:	learn: 7.9088429	test: 12.7953820	best: 12.7953820 (9)	total: 1.74s	remaining: 2m 52s
10:	learn: 7.7110890	test: 12.7153741	best: 12.7153741 (10)	total: 1.92s	remai

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.180254
0:	learn: 16.1263692	test: 18.2382800	best: 18.2382800 (0)	total: 191ms	remaining: 3m 10s
1:	learn: 14.2199679	test: 15.7317163	best: 15.7317163 (1)	total: 422ms	remaining: 3m 30s
2:	learn: 12.7644919	test: 13.3984216	best: 13.3984216 (2)	total: 652ms	remaining: 3m 36s
3:	learn: 11.6298185	test: 11.8711968	best: 11.8711968 (3)	total: 876ms	remaining: 3m 38s
4:	learn: 10.7623031	test: 10.6047967	best: 10.6047967 (4)	total: 1.12s	remaining: 3m 43s
5:	learn: 10.1289391	test: 9.7607824	best: 9.7607824 (5)	total: 1.34s	remaining: 3m 41s
6:	learn: 9.6503783	test: 9.1702616	best: 9.1702616 (6)	total: 1.53s	remaining: 3m 37s
7:	learn: 9.2602350	test: 8.6908637	best: 8.6908637 (7)	total: 1.79s	remaining: 3m 41s
8:	learn: 8.9804136	test: 8.3506839	best: 8.3506839 (8)	total: 2.04s	remaining: 3m 45s
9:	learn: 8.6575204	test: 8.1680570	best: 8.1680570 (9)	total: 2.3s	remaining: 3m 47s
10:	learn: 8.4971941	test: 8.0369287	best: 8.0369287 (10)	total: 2.51s	remaining: 3m 

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.186681
0:	learn: 16.4748216	test: 51.4421430	best: 51.4421430 (0)	total: 309ms	remaining: 5m 8s
1:	learn: 14.3797219	test: 48.6987004	best: 48.6987004 (1)	total: 617ms	remaining: 5m 8s
2:	learn: 12.7812853	test: 46.5182230	best: 46.5182230 (2)	total: 926ms	remaining: 5m 7s
3:	learn: 11.5625219	test: 44.4885448	best: 44.4885448 (3)	total: 1.15s	remaining: 4m 45s
4:	learn: 10.6565114	test: 43.0480841	best: 43.0480841 (4)	total: 1.45s	remaining: 4m 47s
5:	learn: 9.9727511	test: 41.9004079	best: 41.9004079 (5)	total: 1.72s	remaining: 4m 44s
6:	learn: 9.4748318	test: 40.9217015	best: 40.9217015 (6)	total: 1.96s	remaining: 4m 37s
7:	learn: 9.0929561	test: 40.4322784	best: 40.4322784 (7)	total: 2.24s	remaining: 4m 37s
8:	learn: 8.7415505	test: 39.9431778	best: 39.9431778 (8)	total: 2.59s	remaining: 4m 45s
9:	learn: 8.5127455	test: 39.7481827	best: 39.7481827 (9)	total: 2.93s	remaining: 4m 49s
10:	learn: 8.3511137	test: 39.3077699	best: 39.3077699 (10)	total: 3.28s	remai

In [102]:
for i in metrics.keys():
    plt.title(i)
    plt.xlabel('fold')
    pd.Series(metrics[i]).plot(kind='bar')
    plt.show()

In [103]:
metrics

{'R2': [0.7467067727851017,
  0.738635997722974,
  0.8341619782027103,
  0.45642085638444785,
  0.8269973335184851],
 'WAPE': [0.4467182695365406,
  0.4098262643341299,
  0.4132230974431865,
  0.5108860599972064,
  0.4123698416727923],
 'BIAS': [0.126249050858801,
  -0.2628515477155136,
  0.05358902441728794,
  -1.1681582904741477,
  -0.0012367414817186578],
 'MAE': [2.478571401303497,
  2.7026688289131267,
  2.502639015738281,
  3.8495722393225384,
  2.4554250843534136]}

Не самые выдающиеся метрики,но и не самые выдающиеся данные

catboost показывает нам графики.  
pools обеспечивают нам валидацию ошибок трейн и теста  
Видно,что мы не допустили недообучения и переобучения,что важно.  
После каждого фолда мы записываем ранг важности каждого признака,складываем и усредняем.  
Получаем ранги важностей признаков.  
Давайте взглянем  

In [104]:
all_importances = pd.DataFrame(all_importances.groupby('name')['index'].mean().sort_values(ascending=False))
plt.bar(feature_importances['name'], feature_importances['w'])
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [105]:
all_importances.head(10)

,index
name,
sales_last_1_weeks,34.000
quantity_lag_14,32.600
quantity_lag_7,32.200
price_base,29.400
sales_last_4_weeks,29.000
day_of_week_avg_sales,27.400
sales_last_2_weeks,26.800
quantity_lag_30,26.600
day_of_week,24.400


Возьмем топ20 фичей и добавим оставшиеся важные

In [106]:
selected_features = all_importances.head(20).index.to_list()

Лучшие фичи

In [107]:
selected_features

['sales_last_1_weeks',
 'quantity_lag_14',
 'quantity_lag_7',
 'price_base',
 'sales_last_4_weeks',
 'day_of_week_avg_sales',
 'sales_last_2_weeks',
 'quantity_lag_30',
 'day_of_week',
 'dept_name',
 'sales_last_3_weeks',
 'discount_price_perc',
 'date',
 'week_of_year',
 'quarter_avg_sales',
 'discount_price_abs',
 'number_disc_day',
 'weight_volume',
 'day_avg_sales',
 'item_id']

In [108]:
selected_features = selected_features + ['store_id']

обновим инфо об категориальных колонках

Возьмем только нужные фичи далее

In [112]:
train.to_parquet('train')
pd.DataFrame(y).to_parquet('y')
test.to_parquet('test')


In [114]:
train = train[selected_features]
# y = уже в оперативке
y_test = test['quantity']
test = test[selected_features]

In [117]:
categorycal_cols = train.select_dtypes(include=['object']).columns.to_list()

# Подбор параметров

Воспользуемся библиотекой optuna.
Минимизуем rmse

In [119]:

def catboost_objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 500, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0, 5),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_float('random_strength', 0, 2),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 5),
        
    }
    tscv = TimeSeriesSplit(n_splits=2)
    rmse_scores = []
    for _, (train_index, test_index) in enumerate(tscv.split(train)):
        train_train, train_test = train.iloc[train_index], train.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        split_idx = int(len(train_train) * (1 - val_ratio))
        train_train, train_val = train_train.iloc[:split_idx], train_train.iloc[split_idx:]
        y_train, y_val = y_train.iloc[:split_idx], y_train.iloc[split_idx:]
        model = catboost.CatBoostRegressor(**params,
                                random_seed=313,
                                   cat_features=categorycal_cols,
                                   loss_function='RMSE',
                                   early_stopping_rounds=50,
                                   verbose=False)
        model.fit(train_train, y_train , eval_set=(train_val, y_val),plot=False)
        y_pred = model.predict(train_test)
        rmse = (mean_squared_error(y_test, y_pred))**2
        rmse_scores.append(rmse)
    print(f'Trial {trial.number}: {rmse_scores} , mean : {np.mean(rmse_scores)}')
    print('rmse_scores',rmse_scores)
    return np.mean(rmse_scores) 

catboost_study = optuna.create_study(direction='minimize')
catboost_study.optimize(catboost_objective, n_trials=10)

catboost_combined_data_best_params = catboost_study.best_params

[I 2025-01-27 22:40:33,316] A new study created in memory with name: no-name-9a15a475-cc94-4dc0-ab19-2691378d3370
[I 2025-01-27 22:41:30,262] Trial 0 finished with value: 551408.7480905324 and parameters: {'iterations': 756, 'learning_rate': 0.040271775840387776, 'depth': 8, 'l2_leaf_reg': 4.528626714590712, 'border_count': 153, 'random_strength': 1.1839166296410168, 'bagging_temperature': 0.10991332465664416}. Best is trial 0 with value: 551408.7480905324.


Trial 0: [21911.819565984217, 1080905.6766150806] , mean : 551408.7480905324
rmse_scores [21911.819565984217, 1080905.6766150806]


[I 2025-01-27 22:41:53,445] Trial 1 finished with value: 569776.6087918523 and parameters: {'iterations': 2971, 'learning_rate': 0.16530862274168154, 'depth': 3, 'l2_leaf_reg': 3.439367596058628, 'border_count': 111, 'random_strength': 1.6201242355262917, 'bagging_temperature': 3.52470022841503}. Best is trial 0 with value: 551408.7480905324.


Trial 1: [23825.28662457973, 1115727.9309591248] , mean : 569776.6087918523
rmse_scores [23825.28662457973, 1115727.9309591248]


[I 2025-01-27 22:42:29,189] Trial 2 finished with value: 556125.5539288425 and parameters: {'iterations': 4480, 'learning_rate': 0.18217034214062058, 'depth': 8, 'l2_leaf_reg': 0.724788611934804, 'border_count': 112, 'random_strength': 0.9400451543181234, 'bagging_temperature': 3.2430020559395456}. Best is trial 0 with value: 551408.7480905324.


...

Trial 7: [15544.122051359542, 972560.4712189598] , mean : 494052.29663515964
rmse_scores [15544.122051359542, 972560.4712189598]


[I 2025-01-27 22:46:03,134] Trial 8 finished with value: 562037.0067317621 and parameters: {'iterations': 1025, 'learning_rate': 0.22440451155992233, 'depth': 7, 'l2_leaf_reg': 2.3364299592012054, 'border_count': 166, 'random_strength': 1.2413347219088877, 'bagging_temperature': 3.1347614696388644}. Best is trial 7 with value: 494052.29663515964.


Trial 8: [28437.015456394965, 1095636.9980071292] , mean : 562037.0067317621
rmse_scores [28437.015456394965, 1095636.9980071292]


[I 2025-01-27 22:46:23,577] Trial 9 finished with value: 624820.5065228426 and parameters: {'iterations': 3400, 'learning_rate': 0.24883276953420916, 'depth': 4, 'l2_leaf_reg': 1.2190130416292289, 'border_count': 114, 'random_strength': 0.7218327113462595, 'bagging_temperature': 4.8768384083589265}. Best is trial 7 with value: 494052.29663515964.


Trial 9: [26549.64912425124, 1223091.363921434] , mean : 624820.5065228426
rmse_scores [26549.64912425124, 1223091.363921434]


Лучшие параметры катбуста на 10 итерациях поиска

In [120]:
catboost_combined_data_best_params

{'iterations': 3712,
 'learning_rate': 0.23767041503296166,
 'depth': 6,
 'l2_leaf_reg': 2.173241439531821,
 'border_count': 225,
 'random_strength': 0.970245514151574,
 'bagging_temperature': 2.1055292694852983}

# Финальная модель

Обучим итоговую модель на лучших фичах и параметрах

In [126]:
train_train, train_val, y_train, y_val = train_test_split(train, y, test_size=0.2, shuffle=False)# нельзя перемешивать

In [127]:
model  = catboost.CatBoostRegressor(**catboost_combined_data_best_params,
                                random_seed=313,
                                   cat_features=categorycal_cols,
                                   loss_function='RMSE',
                                   early_stopping_rounds=50,
                                   verbose=False)
model.fit(train_train, y_train , eval_set=(train_val, y_val),plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# Предсказание теста

дата задача для каждой точки теста в разрезе продукт-магазин-дата дать прогноз на 7 дней вперед  
Для этого расчитаем базовые предсказания

In [129]:
test['predict'] = model.predict(test)

для обьективного расчета метрики необходимо конечно знать меру в чем продается товар.(вопрос округления в какую сторону и нужно ли)
Но таких данных у нас нет.  
Поэтому оставлю прогноз как есть

In [132]:
y_test

0        3.000
1        2.000
2        2.000
3        3.000
4        4.000
          ... 
413002   2.000
413003   1.000
413004   1.000
413005   2.000
413006   1.000
Name: quantity, Length: 413007, dtype: float64

Вернем обратно таргет для расчета метрик

In [136]:
test['quantity'] = y_test

Посчитаем по каждому артикулу-магазину-и дате на 7 дней вперед включительно сколько было продаж и сколько спрогнозированно

Что здесь в сущности измеренно

In [155]:
test[['date','item_id','quantity','store_id','cast_1_weeks','fact_1_weeks']].head()

,date,item_id,quantity,store_id,cast_1_weeks,fact_1_weeks
0,2024-08-26,001829cb707d,3.000,1,25.967,26.000
1,2024-08-27,001829cb707d,2.000,1,28.287,24.000
2,2024-08-28,001829cb707d,2.000,1,29.351,24.000
3,2024-08-29,001829cb707d,3.000,1,29.428,26.000
4,2024-08-30,001829cb707d,4.000,1,30.163,27.000


у нас есть прогнозы и продажи для каждой строчки,  
Мы просто ищем по ключу их же,и складываем на 7 дней вперед для каждой строчки

In [145]:
test.head(7)['predict'].sum()

25.967057554854883

Вручную проверил считается верно

In [137]:

for weeks, days in [(1,7)]:
    test = add_sum(test, f'cast_{weeks}_weeks','predict')
    test = add_sum(test, f'fact_{weeks}_weeks','quantity')


Так как тест заканчивается 28 числом а прогноз на 7 дней,то обрежем до 21 числа,чтобы правильно измерить метрики

In [139]:
test_to_compute_metrics = test.loc[test['date']<='2024-09-21'].copy()


# Итоговые метрики

In [140]:
print('wape',calculate_wape(test_to_compute_metrics['cast_1_weeks'],test_to_compute_metrics['fact_1_weeks'])),
print('bias',calculate_bias(test_to_compute_metrics['cast_1_weeks'],test_to_compute_metrics['fact_1_weeks']))
print('mae',mean_absolute_error(test_to_compute_metrics['cast_1_weeks'],test_to_compute_metrics['fact_1_weeks']))


wape 0.18681000029723113
bias 0.4726527849376888
mae 6.416807518422039


Данные метрики можно улучшить за счет новых данных:
- Информации об остатках в магазинах
- Прогноза погоды
- Информации о наших посетителях
- И многое и многое другое.
Так же результат можно улучшить применив техники блендинга и стекинга для разных товарных категорий и магазинов  

Посчитаем эффективность прогнозирования по категориям без учета магазинов  
Воспользуемся АBC анализом

In [157]:
sales['sum'] = sales['quantity'] * sales['price_base']
sales.head()

,date,item_id,store_id,quantity,price_base,sum
0,2022-08-28,001829cb707d,1,7.000,134.760,943.320
1,2022-08-28,001829cb707d,2,1.000,148.000,148.000
2,2022-08-28,0022b986c8f0,1,2.000,59.900,119.800
3,2022-08-28,00274a69c705,2,1.000,35.900,35.900
4,2022-08-28,00274a69c705,3,5.000,35.900,179.500


Посчитаем сумму продаж по каждому продукту и долю каждого продукта

In [159]:
abc_analis = sales.groupby('item_id')['sum'].sum().sort_values(ascending=False).reset_index()
abc_analis

,item_id,sum
0,9a7e315f3f42,333375186.179
1,63161948a95a,195430050.000
2,0973df3ff57f,42112929.171
3,7428830d55b6,38893094.598
4,e7e806fd20a2,35383731.874
...,...,...
28304,95da1d9e3800,0.010
28305,2d34a7e69457,0.010
28306,dd2c54241ca9,0.010
28307,63f36cd1a242,0.010


In [160]:
abc_analis['perc'] = abc_analis['sum'] / abc_analis['sum'].sum()*100
abc_analis.head()

,item_id,sum,perc
0,9a7e315f3f42,333375186.179,5.804
1,63161948a95a,195430050.000,3.402
2,0973df3ff57f,42112929.171,0.733
3,7428830d55b6,38893094.598,0.677
4,e7e806fd20a2,35383731.874,0.616


In [162]:
abc_analis['perc_cumsum']  = abc_analis['perc'].cumsum()
abc_analis.loc[abc_analis['perc_cumsum']>80,'category'] = 'C'
abc_analis.loc[abc_analis['perc_cumsum']<20,'category'] = 'A'
abc_analis = abc_analis.fillna('B')
abc_analis.head()

,item_id,sum,perc,perc_cumsum,category
0,9a7e315f3f42,333375186.179,5.804,5.804,A
1,63161948a95a,195430050.000,3.402,9.206,A
2,0973df3ff57f,42112929.171,0.733,9.939,A
3,7428830d55b6,38893094.598,0.677,10.616,A
4,e7e806fd20a2,35383731.874,0.616,11.232,A


In [ ]:
test_to_compute_metrics['error'] = test_to_compute_metrics['fact_1_weeks'] - test_to_compute_metrics['cast_1_weeks']
fact_error_by_items = test_to_compute_metrics.groupby('item_id')['error'].mean().abs()#ошибка по модулю
abc_analis = abc_analis.merge(fact_error_by_items,on='item_id')


In [176]:
pd.DataFrame(abc_analis.groupby('category')['error'].mean())

,error
category,
A,3.238
B,3.279
C,4.436


Наиболее большая ошибка в категории С.
Причина скорее всего в том,что они имеют неполную,разряженную историю продаж.
В этой категории не до конца  всего понятна динамика,сезонность,тренд и тд.  
Что и показывает важность качественных данных,и важность работы по обогащению и востановлению данных  

Итоги:  
Мы провели работу по предсказанию спроса в разрезе разных магазинов.  
Подготовили данные,поработали над признаками,обучением моделей и получили предсказание спроса на необходимый нам   горизонт.  
Работу выполнил Батутин Андрей   
27.01.2025  